# Feature Engineering

In [79]:
# importing all necessary libraries

# pip install -r requirements.txt

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from datetime import datetime, timedelta
import statsmodels.api as sm
import gc
from pylab import rcParams
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import lightgbm as lgb
import calendar

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Load data

In [80]:
# Reading datasets

train = pd.read_csv('input/train_cleansed.csv')
test = pd.read_csv('input/test.csv')

# Formats

train["product_number"] = train["product_number"].astype(str)
train['date'] = pd.to_datetime(train['date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27715 entries, 0 to 27714
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      27715 non-null  datetime64[ns]
 1   id                        27715 non-null  object        
 2   product_number            27715 non-null  object        
 3   reporterhq_id             27715 non-null  int64         
 4   prod_category             27715 non-null  object        
 5   segment                   27715 non-null  object        
 6   inventory_units           27715 non-null  float64       
 7   sales_units               27715 non-null  float64       
 8   inventory_detrended       27715 non-null  float64       
 9   sales_detrended           27715 non-null  float64       
 10  inventory_residuals       27715 non-null  float64       
 11  sales_residuals           27715 non-null  float64       
 12  inventory_units_cl

In [81]:
# Aux functions for feature engineering of season

def get_season(x):
    if x in [12,1,2]:
        return 0
    elif x in [3,4,5]:
        return 1
    elif x in [6,7,8]:
        return 2
    else:
        return 3

# Train data feature engineering

In [82]:
# Create time featuress

datetimes = pd.to_datetime(train['date'])

train['year_number'] = train['date'].dt.year
train['month_number'] = train['date'].dt.month
train['season_number'] = train['month_number'].apply(lambda x:get_season(x))
train['week_number'] = train['date'].dt.isocalendar().week
train.head()

# Lags features

train['inventory_units_l4'] = train.groupby('product_number')['inventory_units'].shift(4)
train['inventory_units_l12'] = train.groupby('product_number')['inventory_units'].shift(12)
train['inventory_units_l52'] = train.groupby('product_number')['inventory_units'].shift(52)

# Data type

train['type'] = 0
train.columns

Index(['date', 'id', 'product_number', 'reporterhq_id', 'prod_category',
       'segment', 'inventory_units', 'sales_units', 'inventory_detrended',
       'sales_detrended', 'inventory_residuals', 'sales_residuals',
       'inventory_units_cleansed', 'sales_units_cleansed', 'year_number',
       'month_number', 'season_number', 'week_number', 'inventory_units_l4',
       'inventory_units_l12', 'inventory_units_l52', 'type'],
      dtype='object')

In [83]:
# Formatting train

train_df = train.drop(['date', 'inventory_detrended', 'sales_detrended', 'inventory_residuals', 'sales_residuals', 'sales_units', 'inventory_units'], axis=1)
train_df = train_df.rename(columns={"sales_units_cleansed": "sales_units", "inventory_units_cleansed": "inventory_units"})
train_df = train_df.iloc[::-1].ffill().iloc[::-1]
train_df.head()

Index(['id', 'product_number', 'reporterhq_id', 'prod_category', 'segment',
       'inventory_units', 'sales_units', 'year_number', 'month_number',
       'season_number', 'week_number', 'inventory_units_l4',
       'inventory_units_l12', 'inventory_units_l52', 'type'],
      dtype='object')

# Test data feature engineering

In [ ]:
# Get categorical values

categories = train_df[['product_number', 'reporterhq_id', 'prod_category', 'segment']].drop_duplicates()

In [86]:
# Sales Test

sales_proj = train_df.groupby(['product_number', 'reporterhq_id']).nth(-1).reset_index()[['product_number', 'reporterhq_id', 'sales_units']]

# Inv Lags Test

# Optimal approach to be implemented: Calculate rolling inventory lags for future forecast predictions

# Current approach: Use for all inventory lags features the latest inv value

inv_lag_proj = train_df.groupby(['product_number', 'reporterhq_id']).nth(-1).reset_index()[['product_number', 'reporterhq_id', 'inventory_units']]

In [87]:
# Future predictions object

test_df = test
test_df[["date", "product_number"]] = test_df["id"].str.split("-", 1, expand = True)
test_df['date_tmp'] = pd.to_datetime(test_df['date'] + '-0', format='%Y%W-%w')
test_df["year_number"] = test_df["date"].str[:-2]
test_df["week_number"] = test_df["date"].str[4:]
test_df["month_number"] = test_df['date_tmp'].dt.month
test_df['season_number'] = test_df['month_number'].apply(lambda x:get_season(x)) 
test_df = test_df.drop(['date','date_tmp'], axis = 1)

# Add product categories

test_df = test_df.merge(categories, on = 'product_number', how='left')

# Add Sales: Latest sales available

test_df = test_df.merge(sales_proj, on = ['product_number', 'reporterhq_id'], how='left')

# Add Latest inv for projections

test_df = test_df.merge(inv_lag_proj, on = ['product_number', 'reporterhq_id'], how='left')

# Rename to get L1, L2, L3 using last inv units

test_df['inventory_units_l4'] = test_df['inventory_units']
test_df['inventory_units_l12'] = test_df['inventory_units']
test_df['inventory_units_l52'] = test_df['inventory_units']

# Type = 1 | Test

test_df['type'] = 1

,id,product_number,year_number,week_number,month_number,season_number,reporterhq_id,prod_category,segment,sales_units,inventory_units,inventory_units_l4,inventory_units_l12,inventory_units_l52,type
0,202319-105609,105609,2023,19,5,1,24,Goku,Gaming,5.0,54.0,54.0,54.0,54.0,1
1,202319-105609,105609,2023,19,5,1,3,Goku,Gaming,7.0,126.0,126.0,126.0,126.0,1
2,202319-107583,107583,2023,19,5,1,15,Clover,Core,15.0,62.0,62.0,62.0,62.0,1
3,202319-107583,107583,2023,19,5,1,27,Clover,Core,11.0,0.0,0.0,0.0,0.0,1
4,202319-107583,107583,2023,19,5,1,39,Clover,Core,0.0,130.0,130.0,130.0,130.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,202331-9870,9870,2023,31,8,2,15,Doraemon,Gaming,94.0,20.0,20.0,20.0,20.0,1
3831,202331-99687,99687,2023,31,8,2,24,Clover,Core,6.0,41.0,41.0,41.0,41.0,1
3832,202331-99687,99687,2023,31,8,2,3,Clover,Core,34.0,26.0,26.0,26.0,26.0,1
3833,202331-99687,99687,2023,31,8,2,60,Clover,Core,3.0,22.0,22.0,22.0,22.0,1


# Concatenate and encode

In [88]:
# Concatenate data

train_test = pd.concat([train_df, test_df], axis = 0)

train_test['week_number'] = train_test['week_number'].astype(str)
train_test['year_number'] = train_test['year_number'].astype(str)

# Get dummies

train_test = pd.get_dummies(train_test, columns = ['product_number', 'segment', 'prod_category', 'reporterhq_id', 'season_number', 'year_number', 'month_number', 'week_number'])

## Train

In [89]:
# Prpeare data for model

train_output = train_test[train_test['type'] == 0]
train_output = train_output.drop(['type'], axis = 1)
train_output.head()

,id,inventory_units,sales_units,inventory_units_l4,inventory_units_l12,inventory_units_l52,product_number_100674,product_number_101661,product_number_102648,product_number_103635,...,week_number_49,week_number_5,week_number_50,week_number_51,week_number_52,week_number_53,week_number_6,week_number_7,week_number_8,week_number_9
0,202104-10857,52.0,0.0,52.0,52.0,52.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,202105-10857,48.0,8.0,52.0,52.0,52.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,202106-10857,72.0,39.0,52.0,52.0,52.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,202107-10857,138.0,0.0,52.0,52.0,52.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,202108-10857,87.0,69.0,52.0,52.0,52.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Test

In [90]:
test_output = train_test[train_test['type'] == 1]
test_output = test_output.drop(['inventory_units','type'], axis = 1)
test_output.head()

,id,sales_units,inventory_units_l4,inventory_units_l12,inventory_units_l52,product_number_100674,product_number_101661,product_number_102648,product_number_103635,product_number_104622,...,week_number_49,week_number_5,week_number_50,week_number_51,week_number_52,week_number_53,week_number_6,week_number_7,week_number_8,week_number_9
0,202319-105609,5.0,54.0,54.0,54.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,202319-105609,7.0,126.0,126.0,126.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,202319-107583,15.0,62.0,62.0,62.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,202319-107583,11.0,0.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,202319-107583,0.0,130.0,130.0,130.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Write outputs

In [91]:
# Output

train_output.to_csv('input/train_ready.csv', index = False)
test_output.to_csv('input/test_ready.csv', index = False)

# Done